In [6]:
import tensorflow as tf
#from keras import layers
#from keras.models import Model, Sequential
#from keras import backend as K

from tensorflow.keras import layers
from tensorflow.keras.models import Model

In [7]:
# input size: Nonex48x48x3
# output size: Nonex36x36x16
def pre_convblock(x):
    y = layers.Conv2D(filters = 64, kernel_size = (5, 5), padding = "valid", activation = "relu")(x)
    y = layers.Conv2D(filters = 64, kernel_size = (5, 5), padding = "valid", activation = "relu")(y)
    y = layers.Conv2D(filters = 16, kernel_size = (5, 5), padding = "valid", activation = "relu")(y)
    return y

In [8]:
# tensor format: [batch, in_height, in_width, in_channels]
# input size: Nonex36x36x32 (cat: 16+16)
# output size: Nonex32x32x2
# softmax is applied along the channel axis.
def post_convblock(x):
    y = layers.Conv2D(filters = 64, kernel_size=(3, 3), padding = "valid", activation = "relu")(x)
    y = layers.Conv2D(filters = 32, kernel_size=(1, 1), padding = "valid", activation = "relu")(y)
    y = layers.Conv2D(filters = 2, kernel_size=(3, 3), padding = "valid")(y)
    y = layers.Softmax(axis = -1)(y)
    return y

In [13]:
# input: out-of-focus image block A & block B, which denote the same area of the whole picture.
#        each: 48x48x3
# output: 
def fusionnet(inTensor1, inTensor2):
    out1 = pre_convblock(inTensor1)
    out2 = pre_convblock(inTensor2)

    x = layers.Concatenate(axis = -1)([out1, out2])
    y = post_convblock(x)
    
    return y

In [ ]:
def gen_img(y):
    ## crop the input images to the same size as network output.
    inCrop1 = layers.Cropping2D(cropping=((8, 8), (8, 8)))(inTensor1)
    inCrop2 = layers.Cropping2D(cropping=((8, 8), (8, 8)))(inTensor2)
    y1, y2 = tf.split(y, [1, 1], axis = 3)
    ## extend y1&y2 dimension to 3, consistant to color channels
    ##y1 = y[:, :, :, 0]
    ##y2 = y[:, :, :, 1]
    y1 = tf.tile(y1, [1, 1, 1, 3])
    y2 = tf.tile(y2, [1, 1, 1, 3])
    y1 = layers.Multiply()([inCrop1, y1])
    y2 = layers.Multiply()([inCrop2, y2])
    y = layers.Add()([y1, y2])
    return y

In [14]:
img1 = layers.Input(shape=(48, 48, 3))
img2 = layers.Input(shape=(48, 48, 3))
pred = fusionnet(img1, img2)
generator = Model(inputs = [img1, img2], outputs = pred)
generator.summary()

ValueError: Output tensors to a Model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: Tensor("add_1/add:0", shape=(?, 32, 32, 3), dtype=float32)